In [23]:
%load_ext autoreload
%autoreload 2
%cd ..
import toolkit as ftk

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/workspaces


In [46]:
import pandas as pd
#.freqstr Q-DEC, M, Y-DEC, W-SUN, D (no B)
a = pd.period_range("2018-01-01", periods=5, freq="W")

PeriodIndex(['2018-01-01/2018-01-07', '2018-01-08/2018-01-14',
             '2018-01-15/2018-01-21', '2018-01-22/2018-01-28',
             '2018-01-29/2018-02-04'],
            dtype='period[W-SUN]')

In [ ]:
import pandas as pd
p = pd.Series([1, 1.25, 1, 0.8, 0.88], index=pd.date_range("2018-01-01", periods=5, freq="W"))
r = pd.Series([0.25, -0.2, -0.2, 0.1], index=pd.period_range("2018-01-08", periods=4, freq="W"))

def p2r(p):
    s = p.pct_change()
    s.index = p.index.to_period()
    return s.dropna()

def r2p(r):
    s = pd.concat([pd.Series([0]), r])
    s.index = pd.date_range(end=r.index[-1].to_timestamp(how='e'), periods=len(r.index) + 1, freq=r.index.freq).date
    return (s + 1).cumprod()

isinstance(r.index, pd.PeriodIndex)

def requireReturn(s, *args, **kwargs):
    def decorator(func):
        def wrapper():
            if isinstance(s.index, pd.DatetimeIndex):
                s = p2r(s)
        return func(s, args, **kwargs)
    return decorator    

@requireReturn
def test(s):
    print(s)

test(p)

2018-01-08/2018-01-14    0.25
2018-01-15/2018-01-21   -0.20
2018-01-22/2018-01-28   -0.20
2018-01-29/2018-02-04    0.10
Freq: W-SUN, dtype: float64
